# 第1スクリーニング: 医療・介護・通信・運送の４分野の技術を保有しており、各分野において活躍している企業を選定する

1. 日経テレコンにおいて、それぞれの分野のキーワードで全期間で記事検索を行い、記事を収集する  
2. 収集した記事について、テキストマイニングを行い、そこから東証上場企業名を抜き出し、ワードクラウドを作成する  
3. 全ての形態素の組み合わせについて、jaccard係数を計算する  
4. 各分野の企業名のうち、出現回数が上位20位の企業のほとんどについては、日経NEEDSの分野分けに含まれているため、スクリーニングの結果に入れる  
5. jaccard係数の単語の組み合わせの片方が分野の関連用語、あるいは、上の出現回数が上位20位の企業である時、その単語ともう片方の単語を結果に入れる
6. 以上の作業で残った単語について、重複しているものと企業名ではないものを除いて、第1スクリーニングとする

### 医療

In [1]:
cd C:\Users\shintaro\stock_league\textmining-care\

C:\Users\shintaro\stock_league\textmining-care


In [2]:
import docx2txt

In [3]:
#テキストの読み込み
#扱うファイル数に合わせて、使用しないコードは # を使うか、 "3つで間を囲むことでコメントアウトしてください

text1 = docx2txt.process('C:/Users/shintaro/StockLeague/stock_league/textmining-care/textmining-c1.docx') #ファイルの名前は各自で決定
text1 = text1.replace('\n','').replace('\u3000','').replace('\xa0','')
#一度textと打ち、ファイルの内容を表示して、余分な文字列があれば、その文字列を上のreplace('','')の最初の''の中に入れると一括で消去される
"""
text2 = docx2txt.process('textmining-m2.docx') #ファイルの名前は各自で決定
text2 = text2.replace('\n','').replace('\u3000','').replace('\xa0','')

text3 = docx2txt.process('textmining-m3.docx') #ファイルの名前は各自で決定
text3 = text3.replace('\n','').replace('\u3000','').replace('\xa0','')


text4 = docx2txt.process('textmining-m4.docx') #ファイルの名前は各自で決定
text4 = text4.replace('\n','').replace('\u3000','').replace('\xa0','')

text5 = docx2txt.process('textmining-m5.docx') #ファイルの名前は各自で決定
text5 = text5.replace('\n','').replace('\u3000','').replace('\xa0','')

text6 = docx2txt.process('textmining-m6.docx') #ファイルの名前は各自で決定
text6 = text6.replace('\n','').replace('\u3000','').replace('\xa0','')

text7 = docx2txt.process('textmining-m7.docx') #ファイルの名前は各自で決定
text7 = text7.replace('\n','').replace('\u3000','').replace('\xa0','')

text8 = docx2txt.process('textmining-m8.docx') #ファイルの名前は各自で決定
text8 = text8.replace('\n','').replace('\u3000','').replace('\xa0','')

text9 = docx2txt.process('textmining-m9.docx') #ファイルの名前は各自で決定
text9 = text9.replace('\n','').replace('\u3000','').replace('\xa0','')
"""

"\ntext2 = docx2txt.process('textmining-m2.docx') #ファイルの名前は各自で決定\ntext2 = text2.replace('\n','').replace('\u3000','').replace('\xa0','')\n\ntext3 = docx2txt.process('textmining-m3.docx') #ファイルの名前は各自で決定\ntext3 = text3.replace('\n','').replace('\u3000','').replace('\xa0','')\n\n\ntext4 = docx2txt.process('textmining-m4.docx') #ファイルの名前は各自で決定\ntext4 = text4.replace('\n','').replace('\u3000','').replace('\xa0','')\n\ntext5 = docx2txt.process('textmining-m5.docx') #ファイルの名前は各自で決定\ntext5 = text5.replace('\n','').replace('\u3000','').replace('\xa0','')\n\ntext6 = docx2txt.process('textmining-m6.docx') #ファイルの名前は各自で決定\ntext6 = text6.replace('\n','').replace('\u3000','').replace('\xa0','')\n\ntext7 = docx2txt.process('textmining-m7.docx') #ファイルの名前は各自で決定\ntext7 = text7.replace('\n','').replace('\u3000','').replace('\xa0','')\n\ntext8 = docx2txt.process('textmining-m8.docx') #ファイルの名前は各自で決定\ntext8 = text8.replace('\n','').replace('\u3000','').replace('\xa0','')\n\ntext9 = docx2txt.process('textmining

In [4]:
#各モジュールのインポート
import csv   
import MeCab
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt 
from pathlib import Path 
import mojimoji
import unicodedata
import urllib
import pandas as pd
import numpy as np
import seaborn as sns
import MeCab
from gensim import corpora
from gensim import matutils
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

data_dir_path = Path('data')

In [5]:
#文を句点で分割するためのモジュールのインポート
import functools

from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation


split_punc2 = functools.partial(split_punctuation, punctuations = r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule = r"^(?P<result>.+)(て)$", remove_former_matched = False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

In [6]:
"""
東証上場企業名が書かれたcsvファイルをエクセルで作成し、
そのファイルを読み込んで企業名をリストwords_kigyouに格納する
"""
with open("C:/Users/shintaro/StockLeague/stock_league/kigyou.csv", encoding = "utf-8-sig") as f:
    reader = csv.reader(f)
    l = [row for row in reader]
    
words_kigyou_tmp = l[0]
words_kigyou = []

for i in range(len(words_kigyou_tmp)):
    words_kigyou.append(words_kigyou_tmp[i].replace('\u3000', ''))

In [7]:
#分割したものをリストとして保存
list1 = list(segmenter(text1))
"""
list2 = list(segmenter(text2))
list3 = list(segmenter(text3))

list4 = list(segmenter(text4))


list5 = list(segmenter(text5))
list6 = list(segmenter(text6))
list7 = list(segmenter(text7))
list8 = list(segmenter(text8))
list9 = list(segmenter(text9))
"""

'\nlist2 = list(segmenter(text2))\nlist3 = list(segmenter(text3))\n\nlist4 = list(segmenter(text4))\n\n\nlist5 = list(segmenter(text5))\nlist6 = list(segmenter(text6))\nlist7 = list(segmenter(text7))\nlist8 = list(segmenter(text8))\nlist9 = list(segmenter(text9))\n'

In [8]:
#リストの結合（この操作により、一文一文が句点で分割されたものをlistsという一つのリストに格納している）
lists = list1# + list2 + list3 + list4 + list5 + list6 + list7 + list8 + list9

In [13]:
lists

['都内経済、深まる疲弊新常態への経営改革道半ば-都議選2021コロナが問う課題2021/06/26 15:00日経速報ニュース1447文字東京都議選を控えた6月下旬の平日。',
 '昼下がりに東京・渋谷センター街を歩くと、ところどころで空き店舗の白い看板が目を引いた。',
 'ある飲食店ビルでは店の看板の3分の1ほどが取り外されていた。',
 '長引く新型コロナウイルス感染と飲食店を中心とした営業制限要請による閉店増加で、街の色彩は失われつつある。',
 '不動産調査のアットホームラボ(東京・千代田)の50坪(1坪は3.3平方メートル)以下の貸店舗動向調査によると、2020年10月～21年3月の都内主要9地区のテナント募集物件は前年同期比61%増えた。',
 '地区別では渋谷が2倍、池袋が77%増で、全体の募集のうち51%は飲食店可の物件だという。',
 '調査を担当する磐前淳子氏は「初めて見る伸び方だ」と驚く。',
 '「家賃が払えなくなり高い物件を手放す事例が目立つ一方、テークアウトなど業態変更に合わせて小規模な物件を探す動きもある」と話す。',
 'コロナの感染拡大から1年強が過ぎた。',
 '都内の景況は感染直後の急激な落ち込みよりはやや持ち直しているが、厳しい局面が続く。',
 '東京都がまとめている業況が「良い」と回答した中小企業の割合から「悪い」を差し引いた業況指数は5月がマイナス44。',
 'コロナ前の19年5月を13ポイント下回る。',
 '東京商工リサーチの調べでは都内のコロナ関連の経営破綻は25日までで累計375件となったが、全体の倒産件数は例年よりも多くない。',
 '国や自治体が緊急支援策として実施している制度融資や雇用調整助成金の拡充、時短営業に伴う協力金などが下支えしている。',
 'ただ、下支え効果が今後も続くかは不透明だ。',
 '多くの事業者で融資資金は取引代金や人件費の支払い、赤字の補塡に充てられ、制度融資を含めた現在の債務に「過剰感がある」とする中小企業は全体の3分の1に上る。',
 '売り上げが落ち込む状況がさらに長引けば、倒産や廃業が加速する可能性がある。',
 '経済活動の縮小に合わせて雇用環境も悪化が続く。',
 '宿泊・飲食サービス業を中心に求人数が大きく減る一方、解雇など雇い主の都合による離職者は増え続けている

In [45]:
#形態素解析（分割した文において形態素解析を行い、その結果を二次元のリストnounsとして保存する）
length = len(lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-d C:/mecab-ipadic-neologd -Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun
    

In [46]:
nouns

[['都内',
  '経済',
  '疲弊',
  '新常態',
  '経営',
  '改革',
  '道',
  '半ば',
  '都議選',
  '2021',
  'コロナ',
  '課題',
  '2021',
  '06',
  '26',
  '15',
  '00',
  '日',
  '経',
  '速報',
  'ニュース',
  '1447',
  '文字',
  '東京都議選',
  '6月下旬',
  '平日'],
 ['昼下がり', '東京・渋谷', 'センター街', 'ところどころ', '空き', '店舗', '看板', '目'],
 ['飲食店', 'ビル', 'では', '店', '看板', '3分の1'],
 ['新型コロナウイルス', '感染', '飲食店', '中心', '営業', '制限', '要請', '閉店', '増加', '街', '色彩'],
 ['不動産',
  '調査',
  'アットホーム',
  'ラボ',
  '東京・千代田',
  '50',
  '坪',
  '1',
  '坪',
  '3',
  '3',
  '平方メートル',
  '以下',
  '貸',
  '店舗',
  '動向',
  '調査',
  '2020年',
  '10月',
  '21年',
  '3月',
  '都内',
  '主要',
  '9',
  '地区',
  'テナント',
  '募集',
  '物件',
  '前年同期',
  '比',
  '61%'],
 ['地区',
  '別',
  '渋谷',
  '2倍',
  '池袋',
  '77%',
  '増',
  '全体',
  '募集',
  'うち',
  '51%',
  '飲食店',
  '可',
  '物件'],
 ['調査', '担当', '磐前', '淳子', '氏', '伸び', '方'],
 ['家賃', '高い', '物件', '事例', '一方', 'テークアウト', '業態', '変更', '小規模', '物件', '動き'],
 ['コロナ', '感染拡大', '1年', '強'],
 ['都内', '景況', '感染', '直後', '急激', '落ち込み', '持ち直している', '局面'],
 ['東京都', '業況', '回答

In [47]:
len_n = len(nouns)

In [48]:
nouns_com = [[] for i in range(len_n)]

In [49]:
#分野の関連用語を入力してください
words_field = ['スーパーシティ', 'DX', 'デジタルトランスフォーメーション', 'モビリティ', '自動車', '運送', 'AI', '医療', '介護', 'ロボット', '遠隔医療']

In [50]:
#処理の軽量化のため、リストの要素のうち必要なもの以外を削除する
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_kigyou:
            nouns_com[i].append(nouns[i][j])
            
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_field:
            nouns_com[i].append(nouns[i][j])

In [51]:
#MeCabで形態素解析を行う
tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text1)
words_wakati1_1 = str_output.split(' ') #元のテキストを分かち書きしたものをwords_wakati1_1に格納する
"""
tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text2)
words_wakati1_2 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text3)
words_wakati1_3 = str_output.split(' ')


tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text4)
words_wakati1_4 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text5)
words_wakati1_5 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text6)
words_wakati1_6 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text7)
words_wakati1_7 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text8)
words_wakati1_8 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text9)
words_wakati1_9 = str_output.split(' ')
"""

'\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text2)\nwords_wakati1_2 = str_output.split(\' \')\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text3)\nwords_wakati1_3 = str_output.split(\' \')\n\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text4)\nwords_wakati1_4 = str_output.split(\' \')\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text5)\nwords_wakati1_5 = str_output.split(\' \')\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text6)\nwords_wakati1_6 = str_output.split(\' \')\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text7)\nwords_wakati1_7 = str_output.split(\' \')\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text8)\nwords_wakati1_8 = str_output.split(\' \')\n\ntagger = MeCab.Tagger("-Owakati")\nstr_output = tagger.parse(text9)\nwords_wakati1_9 = str_output.split(\' \')\n'

In [52]:
#words_wakati_sup = words_wakati1_1 + words_wakati1_2 + words_wakati1_3 + words_wakati1_4 + words_wakati1_5 + words_wakati1_6 + words_wakati1_7 + words_wakati1_8 + words_wakati1_9

In [54]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_1 = len(words_wakati1_1)
"""
n_2 = len(words_wakati1_2)
n_3 = len(words_wakati1_3)

n_4 = len(words_wakati1_4)

n_5 = len(words_wakati1_5)
n_6 = len(words_wakati1_6)
n_7 = len(words_wakati1_7)
n_8 = len(words_wakati1_8)
n_9 = len(words_wakati1_9)
"""

'\nn_2 = len(words_wakati1_2)\nn_3 = len(words_wakati1_3)\n\nn_4 = len(words_wakati1_4)\n\nn_5 = len(words_wakati1_5)\nn_6 = len(words_wakati1_6)\nn_7 = len(words_wakati1_7)\nn_8 = len(words_wakati1_8)\nn_9 = len(words_wakati1_9)\n'

In [55]:
"""
words_wakati1から要素を一つずつ順番に取り出して、その要素がwords_kigyouに含まれているのならば、
新しいリストwords_wakati2にその要素を格納するという作業をwords_wakati1の要素全てに行うことで、
words_wakati2には企業名のみが入ることになる
"""
words_wakati2_1 = []
for i in range(n_1):
    if words_wakati1_1[i] in words_kigyou:
            words_wakati2_1.append(words_wakati1_1[i])
            
words_wakati3_1 = ' '.join(words_wakati2_1)
"""
words_wakati2_2 = []
for i in range(n_2):
    if words_wakati1_2[i] in words_kigyou:
            words_wakati2_2.append(words_wakati1_2[i])
            
words_wakati3_2 = ' '.join(words_wakati2_2)

words_wakati2_3 = []
for i in range(n_3):
    if words_wakati1_3[i] in words_kigyou:
            words_wakati2_3.append(words_wakati1_3[i])
            
words_wakati3_3 = ' '.join(words_wakati2_3)


words_wakati2_4 = []
for i in range(n_4):
    if words_wakati1_4[i] in words_kigyou:
            words_wakati2_4.append(words_wakati1_4[i])
            
words_wakati3_4 = ' '.join(words_wakati2_4)

words_wakati2_5 = []
for i in range(n_5):
    if words_wakati1_5[i] in words_kigyou:
            words_wakati2_5.append(words_wakati1_5[i])
            
words_wakati3_5 = ' '.join(words_wakati2_5)

words_wakati2_6 = []
for i in range(n_6):
    if words_wakati1_6[i] in words_kigyou:
            words_wakati2_6.append(words_wakati1_6[i])
            
words_wakati3_6 = ' '.join(words_wakati2_6)

words_wakati2_7 = []
for i in range(n_7):
    if words_wakati1_7[i] in words_kigyou:
            words_wakati2_7.append(words_wakati1_7[i])
            
words_wakati3_7 = ' '.join(words_wakati2_7)

words_wakati2_8 = []
for i in range(n_8):
    if words_wakati1_8[i] in words_kigyou:
            words_wakati2_8.append(words_wakati1_8[i])
            
words_wakati3_8 = ' '.join(words_wakati2_8)

words_wakati2_9 = []
for i in range(n_9):
    if words_wakati1_9[i] in words_kigyou:
            words_wakati2_9.append(words_wakati1_9[i])
            
words_wakati3_9 = ' '.join(words_wakati2_9)
"""

"\nwords_wakati2_2 = []\nfor i in range(n_2):\n    if words_wakati1_2[i] in words_kigyou:\n            words_wakati2_2.append(words_wakati1_2[i])\n            \nwords_wakati3_2 = ' '.join(words_wakati2_2)\n\nwords_wakati2_3 = []\nfor i in range(n_3):\n    if words_wakati1_3[i] in words_kigyou:\n            words_wakati2_3.append(words_wakati1_3[i])\n            \nwords_wakati3_3 = ' '.join(words_wakati2_3)\n\n\nwords_wakati2_4 = []\nfor i in range(n_4):\n    if words_wakati1_4[i] in words_kigyou:\n            words_wakati2_4.append(words_wakati1_4[i])\n            \nwords_wakati3_4 = ' '.join(words_wakati2_4)\n\nwords_wakati2_5 = []\nfor i in range(n_5):\n    if words_wakati1_5[i] in words_kigyou:\n            words_wakati2_5.append(words_wakati1_5[i])\n            \nwords_wakati3_5 = ' '.join(words_wakati2_5)\n\nwords_wakati2_6 = []\nfor i in range(n_6):\n    if words_wakati1_6[i] in words_kigyou:\n            words_wakati2_6.append(words_wakati1_6[i])\n            \nwords_wakati3_6 =

In [56]:
#リストの結合
words_wakati_WordCloud = words_wakati3_1# + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9

#通信分野の場合は、おそらく下記のコードを上の代わりに使うことになると思います
#words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14 + words_wakati3_15 + words_wakati3_16 + words_wakati3_17 + words_wakati3_18 + words_wakati3_19 + words_wakati3_20 + words_wakati3_21 + words_wakati3_22 + words_wakati3_23 + words_wakati3_24 + words_wakati3_25 + words_wakati3_26 + words_wakati3_27 + words_wakati3_28 + words_wakati3_29 + words_wakati3_30 + words_wakati3_31 + words_wakati3_32 + words_wakati3_33 + words_wakati3_34 + words_wakati3_35 + words_wakati3_36 + words_wakati3_37 + words_wakati3_38 + words_wakati3_39 + words_wakati3_40 + words_wakati3_41 + words_wakati3_42 + words_wakati3_43 + words_wakati3_44 + words_wakati3_45 + words_wakati3_46

In [45]:
cd /Users/shintaro/Downloads/第1スクリーニング

/Users/shintaro/Downloads/第1スクリーニング


In [58]:
#WordCloudを出力したい場合は最終行の#を消去してください
stop_words = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース', 'いつも'] #wordcloudの結果に入れたくないwordを決定する
fpath = "C:\Windows\Fonts\BIZ-UDGothicR.ttc"  # 日本語フォント指定
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    prefer_horizontal=1,
    colormap = 'tab20',
    max_words = 500,   # default=200
    min_font_size = 4,   #default=4
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

wordcloud.to_file("/Users/shintaro/Downloads/WordCloud_医療.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [59]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [83]:
len(list(set(words_wakati_WordCloud.split())))

84

In [90]:
len(list(set(words_wakati_WordCloud1)))

73

In [89]:
words_wakati_WordCloud1

['ニッキ',
 'パナソニック',
 'パナソニック',
 'パナソニック',
 'パナソニック',
 'パナソニック',
 'パナソニック',
 '東急',
 'フェイス',
 '味の素',
 'イオン',
 'ＴＩＳ',
 '東レ',
 '味の素',
 'イオン',
 '花王',
 '花王',
 '花王',
 '花王',
 'クエスト',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '味の素',
 '味の素',
 '味の素',
 '味の素',
 '味の素',
 'ステップ',
 '味の素',
 'ステップ',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 '大和',
 'エーザイ',
 'エーザイ',
 '味の素',
 '味の素',
 '味の素',
 '味の素',
 '味の素',
 'ステップ',
 '味の素',
 'ステップ',
 'パナソニック',
 'パナソニック',
 '東急',
 'フェイス',
 '鈴木',
 'エージェント',
 'フリー',
 'パス',
 'パナソニック',
 'ジョルダン',
 'パス',
 'エーザイ',
 'エーザイ',
 'エーザイ',
 '東レ',
 'ファミリー',
 'フレンドリー',
 '東急',
 '東急',
 '東急',
 '東急',
 'フェイス',
 'フェイス',
 'パナソニック',
 'パナソニック',
 'ティーケーピー',
 'メニコン',
 'メニコン',
 'パス',
 'パス',
 'パナソニック',
 'ジョルダン',
 'パス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 'オーエス',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 '花王',
 'パス',
 'パス

In [78]:
#words_wakati_WordCloud1から同一名称の成分を除外したリストwwW_uniqueを作成する
wwW_unique = list(set(words_wakati_WordCloud1))
wwW_unique

['ＴＩＳ',
 'エージェント',
 '丸紅',
 'イオン',
 'ニッセイ',
 'フジ',
 'クリエイト',
 'ＩＨＩ',
 'ＬＩＸＩＬ',
 'ティーケーピー',
 'ズーム',
 'デンヨー',
 'フェイス',
 'ファミリー',
 '平和',
 'ＡＧＣ',
 'プラネット',
 'ブロッコリー',
 'シャープ',
 'マックス',
 'ステップ',
 'ノジマ',
 'メニコン',
 'ハブ',
 'ルネサンス',
 'ＦＲＯＮＴＥＯ',
 'パス',
 'シード',
 '大和',
 'デンソー',
 'リックス',
 '東レ',
 'ぐるなび',
 'パナソニック',
 'ヤマハ',
 'ココカラファイン',
 'ニッキ',
 '富士通',
 '東急',
 '久世',
 'ワールド',
 'アスクル',
 '味の素',
 '花王',
 'エーザイ',
 'ハイパー',
 'ローソン',
 'ショーケース',
 'ぴあ',
 'ジョルダン',
 '鈴木',
 'ファナック',
 'アトム',
 'クエスト',
 'クレディセゾン',
 'キヤノン',
 'マルシェ',
 'ニコン',
 'アウトソーシング',
 'アルファ',
 'ヤマト',
 'リコー',
 'フリー',
 'ティア',
 'リズム',
 'エムスリー',
 'メドピア',
 'ＴＯＴＯ',
 'サンリオ',
 'オーエス',
 'ディップ',
 'バルミューダ',
 'フレンドリー']

In [93]:
# 元のすべてのテキストデータが入ったリストを代入
documents = lists

# 形態素解析
words_list = []
tagger = MeCab.Tagger(' -d C:/mecab-ipadic-neologd -Ochasen')
for sentence in documents:
    sentence_parsed = tagger.parse(sentence)
    words_in_sentence = []
    for line in sentence_parsed.splitlines()[:-1]:
        words_in_sentence.append(line.split('\t')[0])
    words_list.append(words_in_sentence)

# 単語とユニーク番号の辞書を作成
word2int = {}
i = 0
for words in words_list:
    for word in words:
        if word not in word2int:
            word2int[word] = i
            i += 1

# 単語の出現回数をDataFrameで保存
word2int_gs = corpora.Dictionary(words_list)
bow_gs = np.array([matutils.corpus2dense([word2int_gs.doc2bow(words)], num_terms=len(word2int)).T[0] for words in words_list]).astype(np.int64)
bow_gs_df = pd.DataFrame(bow_gs, columns=list(word2int_gs.values()))

display(bow_gs_df)

-  /  00  06  1447  15  2021  26  6月下旬  :  ...  無断転載  如何  有償  All  \
0     1  2   1   1     1   1     2   1     1  1  ...     0   0   0    0   
1     0  0   0   0     0   0     0   0     0  0  ...     0   0   0    0   
2     0  0   0   0     0   0     0   0     0  0  ...     0   0   0    0   
3     0  0   0   0     0   0     0   0     0  0  ...     0   0   0    0   
4     0  0   0   0     0   0     0   0     0  0  ...     0   0   0    0   
...  .. ..  ..  ..   ...  ..   ...  ..   ... ..  ...   ...  ..  ..  ...   
8309  0  0   0   0     0   0     0   0     0  0  ...     0   0   0    0   
8310  0  0   0   0     0   0     0   0     0  0  ...     0   0   0    0   
8311  0  0   0   0     0   0     0   0     0  0  ...     1   0   0    0   
8312  0  0   0   0     0   0     0   0     0  0  ...     0   1   1    0   
8313  0  0   0   0     0   0     0   0     0  0  ...     0   0   0    1   

      Copyrights  Inc  Nikkei  Reserved  Rights  ©  
0              0    0       0         0       0  0  
1              0    0       0         0       0  0  
2              0    0       0         0       0  0  
3              0    0       0         0       0  0  
4              0    0       0         0       0  0  
...          ...  ...     ...       ...     ... ..  
8309           0    0       0         0       0  0  
8310           0    0       0         0       0  0  
8311           0    0       0         0       0  0  
8312           0    0       0         0       0  0  
8313           1    1       1         1       1  1  

[8314 rows x 16160 columns]

In [94]:
# TFを計算
tf = bow_gs

# IDFを計算
idf = np.log((bow_gs.shape[0] + 1) / (np.sum(bow_gs, axis=0) + 1))

# TF-IDFを計算
tf_idf = tf * (idf + 1)

# TF-IDFをL2正則化
tf_idf_normalized = tf_idf / np.sqrt(np.sum(tf_idf**2, axis=1, keepdims=True))

# 正則化後のTF-IDFをデータフレーム化
tf_idf_normalized_df = pd.DataFrame(tf_idf_normalized, columns=list(word2int_gs.values()))

display(tf_idf_normalized_df)

-         /        00        06      1447        15      2021  \
0     0.148827  0.160847  0.152338  0.111944  0.251348  0.159747  0.214423   
1     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...        ...       ...       ...       ...       ...       ...       ...   
8309  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8310  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8311  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8312  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8313  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

            26      6月下旬         :  ...      無断転載        如何        有償  \
0     0.184424  0.214012  0.100973  ...  0.000000  0.000000  0.000000   
1     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
2     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
4     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
...        ...       ...       ...  ...       ...       ...       ...   
8309  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
8310  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
8311  0.000000  0.000000  0.000000  ...  0.555416  0.000000  0.000000   
8312  0.000000  0.000000  0.000000  ...  0.000000  0.342257  0.342257   
8313  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   

           All  Copyrights       Inc    Nikkei  Reserved    Rights         ©  
0     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
1     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
3     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
4     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
...        ...         ...       ...       ...       ...       ...       ...  
8309  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8310  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8311  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8312  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8313  0.351115    0.351115  0.351115  0.351115  0.351115  0.351115  0.351115  

[8314 rows x 16160 columns]

In [95]:
# scikit-learnを使用した場合
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tfidf_sk = tfidf.fit_transform(bow_gs).toarray()
tfidf_sk_df = pd.DataFrame(tfidf_sk, columns=list(word2int_gs.values()))

display(tfidf_sk_df)

-         /        00       06      1447        15      2021  \
0     0.151894  0.207727  0.150433  0.11372  0.245369  0.159208  0.211876   
1     0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
2     0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
4     0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
...        ...       ...       ...      ...       ...       ...       ...   
8309  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
8310  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
8311  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
8312  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
8313  0.000000  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   

            26      6月下旬         :  ...      無断転載        如何        有償  \
0     0.181156  0.212432  0.117193  ...  0.000000  0.000000  0.000000   
1     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
2     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
4     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
...        ...       ...       ...  ...       ...       ...       ...   
8309  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
8310  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
8311  0.000000  0.000000  0.000000  ...  0.545934  0.000000  0.000000   
8312  0.000000  0.000000  0.000000  ...  0.000000  0.335647  0.335647   
8313  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   

           All  Copyrights       Inc    Nikkei  Reserved    Rights         ©  
0     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
1     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
3     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
4     0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
...        ...         ...       ...       ...       ...       ...       ...  
8309  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8310  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8311  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8312  0.000000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
8313  0.338897    0.338897  0.338897  0.338897  0.338897  0.338897  0.338897  

[8314 rows x 16160 columns]

In [96]:
column = tfidf_sk_df.columns.tolist()

In [ ]:
# 企業名以外の列を削除
length = len(column)

for i in range(length):
    if not column[i] in wwW_unique:
        tfidf_sk_df.drop(column[i], inplace = True, axis = 1)

In [ ]:
tfidf_sk_df.to_csv('C:/Users/shintaro/StockLeague/tfidf.csv', encoding = 'utf-8-sig')

In [ ]:
tfidf_sk_df

In [61]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c1 = c.most_common()

In [62]:
#単語２つの組み合わせを作成
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [63]:
#リスト内の要素の整理
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [64]:
#上で作成したcombination_sentencesは２次元リストなので１次元リストに変換する
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [65]:
combination_sentences = tmp

In [66]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns=['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns=['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [67]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [68]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [69]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [70]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [71]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [72]:
jaccard_coef_data2.to_csv("/Users/shintaro/Downloads/jaccard_coef_医療.csv", index = False, encoding = 'utf-8-sig')

### 介護

In [46]:
cd /Users/shintaro/Downloads/textmining-care

/Users/shintaro/Downloads/textmining-care


In [47]:
#テキストの読み込み
#扱うファイル数に合わせて、使用しないコードは # を使うか、 "3つで間を囲むことでコメントアウトしてください

text1 = docx2txt.process('textmining-c1.docx') #ファイルの名前は各自で決定
text1 = text1.replace('\n','').replace('\u3000','').replace('\xa0','')
#一度textと打ち、ファイルの内容を表示して、余分な文字列があれば、その文字列を上のreplace('','')の最初の''の中に入れると一括で消去される

text2 = docx2txt.process('textmining-c2.docx') #ファイルの名前は各自で決定
text2 = text2.replace('\n','').replace('\u3000','').replace('\xa0','')

text3 = docx2txt.process('textmining-c3.docx') #ファイルの名前は各自で決定
text3 = text3.replace('\n','').replace('\u3000','').replace('\xa0','')


text4 = docx2txt.process('textmining-c4.docx') #ファイルの名前は各自で決定
text4 = text4.replace('\n','').replace('\u3000','').replace('\xa0','')

text5 = docx2txt.process('textmining-c5.docx') #ファイルの名前は各自で決定
text5 = text5.replace('\n','').replace('\u3000','').replace('\xa0','')

text6 = docx2txt.process('textmining-c6.docx') #ファイルの名前は各自で決定
text6 = text6.replace('\n','').replace('\u3000','').replace('\xa0','')

text7 = docx2txt.process('textmining-c7.docx') #ファイルの名前は各自で決定
text7 = text7.replace('\n','').replace('\u3000','').replace('\xa0','')

text8 = docx2txt.process('textmining-c8.docx') #ファイルの名前は各自で決定
text8 = text8.replace('\n','').replace('\u3000','').replace('\xa0','')

text9 = docx2txt.process('textmining-c9.docx') #ファイルの名前は各自で決定
text9 = text9.replace('\n','').replace('\u3000','').replace('\xa0','')


text10 = docx2txt.process('textmining-c10.docx') #ファイルの名前は各自で決定
text10 = text10.replace('\n','').replace('\u3000','').replace('\xa0','')

text11 = docx2txt.process('textmining-c11.docx') #ファイルの名前は各自で決定
text11 = text11.replace('\n','').replace('\u3000','').replace('\xa0','')

In [48]:
#分割したものをリストとして保存
list1 = list(segmenter(text1))
list2 = list(segmenter(text2))
list3 = list(segmenter(text3))

list4 = list(segmenter(text4))


list5 = list(segmenter(text5))
list6 = list(segmenter(text6))
list7 = list(segmenter(text7))
list8 = list(segmenter(text8))
list9 = list(segmenter(text9))
list10 = list(segmenter(text10))
list11 = list(segmenter(text11))

In [49]:
#リストの結合（この操作により、一文一文が句点で分割されたものをlistsという一つのリストに格納している）
lists = list1 + list2 + list3 + list4 + list5 + list6 + list7 + list8 + list9 + list10 + list11

In [50]:
#形態素解析（分割した文において形態素解析を行い、その結果を二次元のリストnounsとして保存する）
length = len(lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun

In [51]:
len_n = len(nouns)

In [52]:
nouns_com = [[] for i in range(len_n)]

In [53]:
#分野の関連用語を入力してください
words_field = ['スーパーシティ', 'DX', 'デジタルトランスフォーメーション', 'モビリティ', '自動車', '運送', 'AI', '医療', '介護', 'ロボット', '遠隔医療']

In [54]:
#処理の軽量化のため、リストの要素のうち必要なもの以外を削除する
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_kigyou:
            nouns_com[i].append(nouns[i][j])
            
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_field:
            nouns_com[i].append(nouns[i][j])

In [55]:
#MeCabで形態素解析を行う
tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text1)
words_wakati1_1 = str_output.split(' ') #元のテキストを分かち書きしたものをwords_wakati1_1に格納する

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text2)
words_wakati1_2 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text3)
words_wakati1_3 = str_output.split(' ')


tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text4)
words_wakati1_4 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text5)
words_wakati1_5 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text6)
words_wakati1_6 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text7)
words_wakati1_7 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text8)
words_wakati1_8 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text9)
words_wakati1_9 = str_output.split(' ')



tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text10)
words_wakati1_10 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text11)
words_wakati1_11 = str_output.split(' ')

In [56]:
#words_wakati_sup = words_wakati1_1 + words_wakati1_2 + words_wakati1_3 + words_wakati1_4 + words_wakati1_5 + words_wakati1_6 + words_wakati1_7 + words_wakati1_8 + words_wakati1_9

In [57]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_1 = len(words_wakati1_1)

n_2 = len(words_wakati1_2)
n_3 = len(words_wakati1_3)

n_4 = len(words_wakati1_4)

n_5 = len(words_wakati1_5)
n_6 = len(words_wakati1_6)
n_7 = len(words_wakati1_7)
n_8 = len(words_wakati1_8)
n_9 = len(words_wakati1_9)

n_10 = len(words_wakati1_10)
n_11 = len(words_wakati1_11)

In [58]:
"""
words_wakati1から要素を一つずつ順番に取り出して、その要素がwords_kigyouに含まれているのならば、
新しいリストwords_wakati2にその要素を格納するという作業をwords_wakati1の要素全てに行うことで、
words_wakati2には企業名のみが入ることになる
"""
words_wakati2_1 = []
for i in range(n_1):
    if words_wakati1_1[i] in words_kigyou:
            words_wakati2_1.append(words_wakati1_1[i])
            
words_wakati3_1 = ' '.join(words_wakati2_1)

words_wakati2_2 = []
for i in range(n_2):
    if words_wakati1_2[i] in words_kigyou:
            words_wakati2_2.append(words_wakati1_2[i])
            
words_wakati3_2 = ' '.join(words_wakati2_2)

words_wakati2_3 = []
for i in range(n_3):
    if words_wakati1_3[i] in words_kigyou:
            words_wakati2_3.append(words_wakati1_3[i])
            
words_wakati3_3 = ' '.join(words_wakati2_3)


words_wakati2_4 = []
for i in range(n_4):
    if words_wakati1_4[i] in words_kigyou:
            words_wakati2_4.append(words_wakati1_4[i])
            
words_wakati3_4 = ' '.join(words_wakati2_4)

words_wakati2_5 = []
for i in range(n_5):
    if words_wakati1_5[i] in words_kigyou:
            words_wakati2_5.append(words_wakati1_5[i])
            
words_wakati3_5 = ' '.join(words_wakati2_5)

words_wakati2_6 = []
for i in range(n_6):
    if words_wakati1_6[i] in words_kigyou:
            words_wakati2_6.append(words_wakati1_6[i])
            
words_wakati3_6 = ' '.join(words_wakati2_6)

words_wakati2_7 = []
for i in range(n_7):
    if words_wakati1_7[i] in words_kigyou:
            words_wakati2_7.append(words_wakati1_7[i])
            
words_wakati3_7 = ' '.join(words_wakati2_7)

words_wakati2_8 = []
for i in range(n_8):
    if words_wakati1_8[i] in words_kigyou:
            words_wakati2_8.append(words_wakati1_8[i])
            
words_wakati3_8 = ' '.join(words_wakati2_8)

words_wakati2_9 = []
for i in range(n_9):
    if words_wakati1_9[i] in words_kigyou:
            words_wakati2_9.append(words_wakati1_9[i])
            
words_wakati3_9 = ' '.join(words_wakati2_9)


words_wakati2_10 = []
for i in range(n_10):
    if words_wakati1_10[i] in words_kigyou:
            words_wakati2_10.append(words_wakati1_10[i])
            
words_wakati3_10 = ' '.join(words_wakati2_10)

words_wakati2_11 = []
for i in range(n_11):
    if words_wakati1_11[i] in words_kigyou:
            words_wakati2_11.append(words_wakati1_11[i])
            
words_wakati3_11 = ' '.join(words_wakati2_11)

In [59]:
#リストの結合
words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11

#通信分野の場合は、おそらく下記のコードを上の代わりに使うことになると思います
#words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14 + words_wakati3_15 + words_wakati3_16 + words_wakati3_17 + words_wakati3_18 + words_wakati3_19 + words_wakati3_20 + words_wakati3_21 + words_wakati3_22 + words_wakati3_23 + words_wakati3_24 + words_wakati3_25 + words_wakati3_26 + words_wakati3_27 + words_wakati3_28 + words_wakati3_29 + words_wakati3_30 + words_wakati3_31 + words_wakati3_32 + words_wakati3_33 + words_wakati3_34 + words_wakati3_35 + words_wakati3_36 + words_wakati3_37 + words_wakati3_38 + words_wakati3_39 + words_wakati3_40 + words_wakati3_41 + words_wakati3_42 + words_wakati3_43 + words_wakati3_44 + words_wakati3_45 + words_wakati3_46

In [60]:
cd /Users/shintaro/Downloads/第1スクリーニング

/Users/shintaro/Downloads/第1スクリーニング


In [66]:
#WordCloudを出力したい場合は最終行の#を消去してください
stop_words = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース', '鈴木'] #wordcloudの結果に入れたくないwordを決定する
fpath = '/Users/shintaro/Library/Fonts//Arial Unicode.ttf'  # 日本語フォント指定
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    max_words = 500, # default=200
    min_font_size = 4,   #default=4
    prefer_horizontal = 1,
    colormap = 'tab20',
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

wordcloud.to_file("WordCloud_介護.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [67]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [68]:
#words_wakati_WordCloud1から同一名称の成分を除外したリストwwW_uniqueを作成する
wwW_unique = list(set(words_wakati_WordCloud1))
#wwW_unique

In [69]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c2 = c.most_common()

In [70]:
#単語２つの組み合わせを作成
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [71]:
#リスト内の要素の整理
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [72]:
#上で作成したcombination_sentencesは２次元リストなので１次元リストに変換する
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [73]:
combination_sentences = tmp

In [74]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns=['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns=['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [75]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [76]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [77]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [78]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [79]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [80]:
jaccard_coef_data2.to_csv("jaccard_coef_介護.csv", index = False, encoding = 'utf-8-sig')

### 運送

In [82]:
cd /Users/shintaro/Downloads/運送

/Users/shintaro/Downloads/運送


In [83]:
#テキストの読み込み
#扱うファイル数に合わせて、使用しないコードは # を使うか、 "3つで間を囲むことでコメントアウトしてください

text1 = docx2txt.process('運送１.docx') #ファイルの名前は各自で決定
text1 = text1.replace('\n','').replace('\u3000','').replace('\xa0','')
#一度textと打ち、ファイルの内容を表示して、余分な文字列があれば、その文字列を上のreplace('','')の最初の''の中に入れると一括で消去される

text2 = docx2txt.process('運送２.docx') #ファイルの名前は各自で決定
text2 = text2.replace('\n','').replace('\u3000','').replace('\xa0','')

text3 = docx2txt.process('運送３.docx') #ファイルの名前は各自で決定
text3 = text3.replace('\n','').replace('\u3000','').replace('\xa0','')

In [84]:
#分割したものをリストとして保存
list1 = list(segmenter(text1))
list2 = list(segmenter(text2))
list3 = list(segmenter(text3))

In [85]:
#リストの結合（この操作により、一文一文が句点で分割されたものをlistsという一つのリストに格納している）
lists = list1 + list2 + list3

In [87]:
#形態素解析（分割した文において形態素解析を行い、その結果を二次元のリストnounsとして保存する）
length = len(lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun

In [88]:
len_n = len(nouns)

In [89]:
nouns_com = [[] for i in range(len_n)]

In [90]:
#分野の関連用語を入力してください
words_field = ['スーパーシティ', 'DX', 'デジタルトランスフォーメーション', 'モビリティ', '自動車', '運送', 'AI', '医療', '介護', 'ロボット', '遠隔医療']

In [91]:
#処理の軽量化のため、リストの要素のうち必要なもの以外を削除する
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_kigyou:
            nouns_com[i].append(nouns[i][j])
            
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_field:
            nouns_com[i].append(nouns[i][j])

In [92]:
#MeCabで形態素解析を行う
tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text1)
words_wakati1_1 = str_output.split(' ') #元のテキストを分かち書きしたものをwords_wakati1_1に格納する

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text2)
words_wakati1_2 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text3)
words_wakati1_3 = str_output.split(' ')

In [93]:
#words_wakati_sup = words_wakati1_1 + words_wakati1_2 + words_wakati1_3 + words_wakati1_4 + words_wakati1_5 + words_wakati1_6 + words_wakati1_7 + words_wakati1_8 + words_wakati1_9

In [94]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_1 = len(words_wakati1_1)

n_2 = len(words_wakati1_2)
n_3 = len(words_wakati1_3)

In [95]:
"""
words_wakati1から要素を一つずつ順番に取り出して、その要素がwords_kigyouに含まれているのならば、
新しいリストwords_wakati2にその要素を格納するという作業をwords_wakati1の要素全てに行うことで、
words_wakati2には企業名のみが入ることになる
"""
words_wakati2_1 = []
for i in range(n_1):
    if words_wakati1_1[i] in words_kigyou:
            words_wakati2_1.append(words_wakati1_1[i])
            
words_wakati3_1 = ' '.join(words_wakati2_1)

words_wakati2_2 = []
for i in range(n_2):
    if words_wakati1_2[i] in words_kigyou:
            words_wakati2_2.append(words_wakati1_2[i])
            
words_wakati3_2 = ' '.join(words_wakati2_2)

words_wakati2_3 = []
for i in range(n_3):
    if words_wakati1_3[i] in words_kigyou:
            words_wakati2_3.append(words_wakati1_3[i])
            
words_wakati3_3 = ' '.join(words_wakati2_3)

In [96]:
#リストの結合
words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3

#通信分野の場合は、おそらく下記のコードを上の代わりに使うことになると思います
#words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14 + words_wakati3_15 + words_wakati3_16 + words_wakati3_17 + words_wakati3_18 + words_wakati3_19 + words_wakati3_20 + words_wakati3_21 + words_wakati3_22 + words_wakati3_23 + words_wakati3_24 + words_wakati3_25 + words_wakati3_26 + words_wakati3_27 + words_wakati3_28 + words_wakati3_29 + words_wakati3_30 + words_wakati3_31 + words_wakati3_32 + words_wakati3_33 + words_wakati3_34 + words_wakati3_35 + words_wakati3_36 + words_wakati3_37 + words_wakati3_38 + words_wakati3_39 + words_wakati3_40 + words_wakati3_41 + words_wakati3_42 + words_wakati3_43 + words_wakati3_44 + words_wakati3_45 + words_wakati3_46

In [98]:
cd /Users/shintaro/Downloads/第1スクリーニング

/Users/shintaro/Downloads/第1スクリーニング


In [101]:
#WordCloudを出力したい場合は最終行の#を消去してください
stop_words = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース', '鈴木'] #wordcloudの結果に入れたくないwordを決定する
fpath = '/Users/shintaro/Library/Fonts//Arial Unicode.ttf'  # 日本語フォント指定
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    max_words = 500,   # default=200
    min_font_size = 4,   #default=4
    prefer_horizontal = 1,
    colormap = 'tab20', 
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

wordcloud.to_file("WordCloud_運送.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [102]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [103]:
#words_wakati_WordCloud1から同一名称の成分を除外したリストwwW_uniqueを作成する
wwW_unique = list(set(words_wakati_WordCloud1))
#wwW_unique

In [104]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c3 = c.most_common()

In [105]:
#単語２つの組み合わせを作成
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [106]:
#リスト内の要素の整理
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [107]:
#上で作成したcombination_sentencesは２次元リストなので１次元リストに変換する
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [108]:
combination_sentences = tmp

In [109]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns=['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns=['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [110]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [111]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [112]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [113]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [114]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [115]:
jaccard_coef_data2.to_csv("jaccard_coef_運送.csv", index = False, encoding = 'utf-8-sig')

### モビリティ

In [116]:
cd /Users/shintaro/Downloads/モビリティ

/Users/shintaro/Downloads/モビリティ


In [117]:
#テキストの読み込み
#扱うファイル数に合わせて、使用しないコードは # を使うか、 "3つで間を囲むことでコメントアウトしてください

text1 = docx2txt.process('モビリティ1.docx') #ファイルの名前は各自で決定
text1 = text1.replace('\n','').replace('\u3000','').replace('\xa0','')
#一度textと打ち、ファイルの内容を表示して、余分な文字列があれば、その文字列を上のreplace('','')の最初の''の中に入れると一括で消去される

text2 = docx2txt.process('モビリティ2.docx') #ファイルの名前は各自で決定
text2 = text2.replace('\n','').replace('\u3000','').replace('\xa0','')

text3 = docx2txt.process('モビリティ3.docx') #ファイルの名前は各自で決定
text3 = text3.replace('\n','').replace('\u3000','').replace('\xa0','')


text4 = docx2txt.process('モビリティ4.docx') #ファイルの名前は各自で決定
text4 = text4.replace('\n','').replace('\u3000','').replace('\xa0','')

In [118]:
#分割したものをリストとして保存
list1 = list(segmenter(text1))
list2 = list(segmenter(text2))
list3 = list(segmenter(text3))

list4 = list(segmenter(text4))

In [119]:
#リストの結合（この操作により、一文一文が句点で分割されたものをlistsという一つのリストに格納している）
lists = list1 + list2 + list3 + list4

In [120]:
#形態素解析（分割した文において形態素解析を行い、その結果を二次元のリストnounsとして保存する）
length = len(lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun

In [121]:
len_n = len(nouns)

In [122]:
nouns_com = [[] for i in range(len_n)]

In [123]:
#分野の関連用語を入力してください
words_field = ['スーパーシティ', 'DX', 'デジタルトランスフォーメーション', 'モビリティ', '自動車', '運送', 'AI', '医療', '介護', 'ロボット', '遠隔医療']

In [124]:
#処理の軽量化のため、リストの要素のうち必要なもの以外を削除する
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_kigyou:
            nouns_com[i].append(nouns[i][j])
            
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_field:
            nouns_com[i].append(nouns[i][j])

In [125]:
#MeCabで形態素解析を行う
tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text1)
words_wakati1_1 = str_output.split(' ') #元のテキストを分かち書きしたものをwords_wakati1_1に格納する

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text2)
words_wakati1_2 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text3)
words_wakati1_3 = str_output.split(' ')


tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text4)
words_wakati1_4 = str_output.split(' ')

In [126]:
#words_wakati_sup = words_wakati1_1 + words_wakati1_2 + words_wakati1_3 + words_wakati1_4 + words_wakati1_5 + words_wakati1_6 + words_wakati1_7 + words_wakati1_8 + words_wakati1_9

In [127]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_1 = len(words_wakati1_1)

n_2 = len(words_wakati1_2)
n_3 = len(words_wakati1_3)

n_4 = len(words_wakati1_4)

In [128]:
"""
words_wakati1から要素を一つずつ順番に取り出して、その要素がwords_kigyouに含まれているのならば、
新しいリストwords_wakati2にその要素を格納するという作業をwords_wakati1の要素全てに行うことで、
words_wakati2には企業名のみが入ることになる
"""
words_wakati2_1 = []
for i in range(n_1):
    if words_wakati1_1[i] in words_kigyou:
            words_wakati2_1.append(words_wakati1_1[i])
            
words_wakati3_1 = ' '.join(words_wakati2_1)

words_wakati2_2 = []
for i in range(n_2):
    if words_wakati1_2[i] in words_kigyou:
            words_wakati2_2.append(words_wakati1_2[i])
            
words_wakati3_2 = ' '.join(words_wakati2_2)

words_wakati2_3 = []
for i in range(n_3):
    if words_wakati1_3[i] in words_kigyou:
            words_wakati2_3.append(words_wakati1_3[i])
            
words_wakati3_3 = ' '.join(words_wakati2_3)


words_wakati2_4 = []
for i in range(n_4):
    if words_wakati1_4[i] in words_kigyou:
            words_wakati2_4.append(words_wakati1_4[i])
            
words_wakati3_4 = ' '.join(words_wakati2_4)

In [129]:
#リストの結合
words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4# + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14

#通信分野の場合は、おそらく下記のコードを上の代わりに使うことになると思います
#words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14 + words_wakati3_15 + words_wakati3_16 + words_wakati3_17 + words_wakati3_18 + words_wakati3_19 + words_wakati3_20 + words_wakati3_21 + words_wakati3_22 + words_wakati3_23 + words_wakati3_24 + words_wakati3_25 + words_wakati3_26 + words_wakati3_27 + words_wakati3_28 + words_wakati3_29 + words_wakati3_30 + words_wakati3_31 + words_wakati3_32 + words_wakati3_33 + words_wakati3_34 + words_wakati3_35 + words_wakati3_36 + words_wakati3_37 + words_wakati3_38 + words_wakati3_39 + words_wakati3_40 + words_wakati3_41 + words_wakati3_42 + words_wakati3_43 + words_wakati3_44 + words_wakati3_45 + words_wakati3_46

In [130]:
cd /Users/shintaro/Downloads/第1スクリーニング

/Users/shintaro/Downloads/第1スクリーニング


In [131]:
#WordCloudを出力したい場合は最終行の#を消去してください
stop_words = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース', '鈴木'] #wordcloudの結果に入れたくないwordを決定する
fpath = '/Users/shintaro/Library/Fonts//Arial Unicode.ttf'  # 日本語フォント指定
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    max_words = 500,   # default=200
    min_font_size = 4,   #default=4
    prefer_horizontal = 1,
    colormap = 'tab20',
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

wordcloud.to_file("WordCloud_モビリティ.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [132]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [133]:
#words_wakati_WordCloud1から同一名称の成分を除外したリストwwW_uniqueを作成する
wwW_unique = list(set(words_wakati_WordCloud1))
#wwW_unique

In [134]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c4 = c.most_common()

In [135]:
#単語２つの組み合わせを作成
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [136]:
#リスト内の要素の整理
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [137]:
#上で作成したcombination_sentencesは２次元リストなので１次元リストに変換する
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [138]:
combination_sentences = tmp

In [139]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns=['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns=['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [140]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [141]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [142]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [143]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [144]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [145]:
jaccard_coef_data2.to_csv("jaccard_coef_モビリティ.csv", index = False, encoding = 'utf-8-sig')

### 通信

In [146]:
cd /Users/shintaro/Downloads/通信

/Users/shintaro/Downloads/通信


In [147]:
#テキストの読み込み
#扱うファイル数に合わせて、使用しないコードは # を使うか、 "3つで間を囲むことでコメントアウトしてください

text1 = docx2txt.process('通信1.docx') #ファイルの名前は各自で決定
text1 = text1.replace('\n','').replace('\u3000','').replace('\xa0','')
#一度textと打ち、ファイルの内容を表示して、余分な文字列があれば、その文字列を上のreplace('','')の最初の''の中に入れると一括で消去される

text2 = docx2txt.process('通信2.docx') #ファイルの名前は各自で決定
text2 = text2.replace('\n','').replace('\u3000','').replace('\xa0','')

text3 = docx2txt.process('通信3.docx') #ファイルの名前は各自で決定
text3 = text3.replace('\n','').replace('\u3000','').replace('\xa0','')


text4 = docx2txt.process('通信4.docx') #ファイルの名前は各自で決定
text4 = text4.replace('\n','').replace('\u3000','').replace('\xa0','')

text5 = docx2txt.process('通信5.docx') #ファイルの名前は各自で決定
text5 = text5.replace('\n','').replace('\u3000','').replace('\xa0','')

text6 = docx2txt.process('通信6.docx') #ファイルの名前は各自で決定
text6 = text6.replace('\n','').replace('\u3000','').replace('\xa0','')

text7 = docx2txt.process('通信7.docx') #ファイルの名前は各自で決定
text7 = text7.replace('\n','').replace('\u3000','').replace('\xa0','')

text8 = docx2txt.process('通信8.docx') #ファイルの名前は各自で決定
text8 = text8.replace('\n','').replace('\u3000','').replace('\xa0','')

text9 = docx2txt.process('通信9.docx') #ファイルの名前は各自で決定
text9 = text9.replace('\n','').replace('\u3000','').replace('\xa0','')


text10 = docx2txt.process('通信10.docx') #ファイルの名前は各自で決定
text10 = text10.replace('\n','').replace('\u3000','').replace('\xa0','')

text11 = docx2txt.process('通信11.docx') #ファイルの名前は各自で決定
text11 = text11.replace('\n','').replace('\u3000','').replace('\xa0','')



text12 = docx2txt.process('通信12.docx') #ファイルの名前は各自で決定
text12 = text12.replace('\n','').replace('\u3000','').replace('\xa0','')

text13 = docx2txt.process('通信13.docx') #ファイルの名前は各自で決定
text13 = text13.replace('\n','').replace('\u3000','').replace('\xa0','')

text14 = docx2txt.process('通信14.docx') #ファイルの名前は各自で決定
text14 = text14.replace('\n','').replace('\u3000','').replace('\xa0','')


text15 = docx2txt.process('通信15.docx') #ファイルの名前は各自で決定
text15 = text15.replace('\n','').replace('\u3000','').replace('\xa0','')

text16 = docx2txt.process('通信16.docx') #ファイルの名前は各自で決定
text16 = text16.replace('\n','').replace('\u3000','').replace('\xa0','')

text17 = docx2txt.process('通信17.docx') #ファイルの名前は各自で決定
text17 = text17.replace('\n','').replace('\u3000','').replace('\xa0','')

text18 = docx2txt.process('通信18.docx') #ファイルの名前は各自で決定
text18 = text18.replace('\n','').replace('\u3000','').replace('\xa0','')

text19 = docx2txt.process('通信19.docx') #ファイルの名前は各自で決定
text19 = text19.replace('\n','').replace('\u3000','').replace('\xa0','')

text20 = docx2txt.process('通信20.docx') #ファイルの名前は各自で決定
text20 = text20.replace('\n','').replace('\u3000','').replace('\xa0','')

text21 = docx2txt.process('通信21.docx') #ファイルの名前は各自で決定
text21 = text21.replace('\n','').replace('\u3000','').replace('\xa0','')

text22 = docx2txt.process('通信22.docx') #ファイルの名前は各自で決定
text22 = text22.replace('\n','').replace('\u3000','').replace('\xa0','')

text23 = docx2txt.process('通信23.docx') #ファイルの名前は各自で決定
text23 = text23.replace('\n','').replace('\u3000','').replace('\xa0','')

text24 = docx2txt.process('通信24.docx') #ファイルの名前は各自で決定
text24 = text24.replace('\n','').replace('\u3000','').replace('\xa0','')

text25 = docx2txt.process('通信25.docx') #ファイルの名前は各自で決定
text25 = text25.replace('\n','').replace('\u3000','').replace('\xa0','')

text26 = docx2txt.process('通信26.docx') #ファイルの名前は各自で決定
text26 = text26.replace('\n','').replace('\u3000','').replace('\xa0','')

text27 = docx2txt.process('通信27.docx') #ファイルの名前は各自で決定
text27 = text27.replace('\n','').replace('\u3000','').replace('\xa0','')

text28 = docx2txt.process('通信28.docx') #ファイルの名前は各自で決定
text28 = text28.replace('\n','').replace('\u3000','').replace('\xa0','')

text29 = docx2txt.process('通信29.docx') #ファイルの名前は各自で決定
text29 = text29.replace('\n','').replace('\u3000','').replace('\xa0','')

text30 = docx2txt.process('通信30.docx') #ファイルの名前は各自で決定
text30 = text30.replace('\n','').replace('\u3000','').replace('\xa0','')

text31 = docx2txt.process('通信31.docx') #ファイルの名前は各自で決定
text31 = text31.replace('\n','').replace('\u3000','').replace('\xa0','')

text32 = docx2txt.process('通信32.docx') #ファイルの名前は各自で決定
text32 = text32.replace('\n','').replace('\u3000','').replace('\xa0','')

text33 = docx2txt.process('通信33.docx') #ファイルの名前は各自で決定
text33 = text33.replace('\n','').replace('\u3000','').replace('\xa0','')

text34 = docx2txt.process('通信34.docx') #ファイルの名前は各自で決定
text34 = text34.replace('\n','').replace('\u3000','').replace('\xa0','')

text35 = docx2txt.process('通信35.docx') #ファイルの名前は各自で決定
text35 = text35.replace('\n','').replace('\u3000','').replace('\xa0','')

text36 = docx2txt.process('通信36.docx') #ファイルの名前は各自で決定
text36 = text36.replace('\n','').replace('\u3000','').replace('\xa0','')

text37 = docx2txt.process('通信37.docx') #ファイルの名前は各自で決定
text37 = text37.replace('\n','').replace('\u3000','').replace('\xa0','')

text38 = docx2txt.process('通信38.docx') #ファイルの名前は各自で決定
text38 = text38.replace('\n','').replace('\u3000','').replace('\xa0','')

text39 = docx2txt.process('通信39.docx') #ファイルの名前は各自で決定
text39 = text39.replace('\n','').replace('\u3000','').replace('\xa0','')

text40 = docx2txt.process('通信40.docx') #ファイルの名前は各自で決定
text40 = text40.replace('\n','').replace('\u3000','').replace('\xa0','')

text41 = docx2txt.process('通信41.docx') #ファイルの名前は各自で決定
text41 = text41.replace('\n','').replace('\u3000','').replace('\xa0','')

text42 = docx2txt.process('通信42.docx') #ファイルの名前は各自で決定
text42 = text42.replace('\n','').replace('\u3000','').replace('\xa0','')

text43 = docx2txt.process('通信43.docx') #ファイルの名前は各自で決定
text43 = text43.replace('\n','').replace('\u3000','').replace('\xa0','')

text44 = docx2txt.process('通信44.docx') #ファイルの名前は各自で決定
text44 = text44.replace('\n','').replace('\u3000','').replace('\xa0','')

text45 = docx2txt.process('通信45.docx') #ファイルの名前は各自で決定
text45 = text45.replace('\n','').replace('\u3000','').replace('\xa0','')

text46 = docx2txt.process('通信46.docx') #ファイルの名前は各自で決定
text46 = text46.replace('\n','').replace('\u3000','').replace('\xa0','')

In [148]:
#分割したものをリストとして保存
list1 = list(segmenter(text1))
list2 = list(segmenter(text2))
list3 = list(segmenter(text3))

list4 = list(segmenter(text4))


list5 = list(segmenter(text5))
list6 = list(segmenter(text6))
list7 = list(segmenter(text7))
list8 = list(segmenter(text8))
list9 = list(segmenter(text9))
list10 = list(segmenter(text10))
list11 = list(segmenter(text11))


list12 = list(segmenter(text12))
list13 = list(segmenter(text13))
list14 = list(segmenter(text14))


list15 = list(segmenter(text15))
list16 = list(segmenter(text16))
list17 = list(segmenter(text17))
list18 = list(segmenter(text18))
list19 = list(segmenter(text19))
list20 = list(segmenter(text20))
list21 = list(segmenter(text21))
list22 = list(segmenter(text22))
list23 = list(segmenter(text23))
list24 = list(segmenter(text24))
list25 = list(segmenter(text25))
list26 = list(segmenter(text26))
list27 = list(segmenter(text27))
list28 = list(segmenter(text28))
list29 = list(segmenter(text29))
list30 = list(segmenter(text30))
list31 = list(segmenter(text31))
list32 = list(segmenter(text32))
list33 = list(segmenter(text33))
list34 = list(segmenter(text34))
list35 = list(segmenter(text35))
list36 = list(segmenter(text36))
list37 = list(segmenter(text37))
list38 = list(segmenter(text38))
list39 = list(segmenter(text39))
list40 = list(segmenter(text40))
list41 = list(segmenter(text41))
list42 = list(segmenter(text42))
list43 = list(segmenter(text43))
list44 = list(segmenter(text44))
list45 = list(segmenter(text45))
list46 = list(segmenter(text46))

In [149]:
#リストの結合（この操作により、一文一文が句点で分割されたものをlistsという一つのリストに格納している）
lists = list1 + list2 + list3 + list4 + list5 + list6 + list7 + list8 + list9 + list10 + list11 + list12 + list13 + list14 + list15 + list16 + list17 + list18 + list19 + list20 + list21 + list22 + list23 + list24 + list25 + list26 + list27 + list28 + list29 + list30 + list31 + list32 + list33 + list34 + list35 + list36 + list37 + list38 + list39 + list40 + list41 + list42 + list43 + list44 + list45 + list46

In [150]:
#形態素解析（分割した文において形態素解析を行い、その結果を二次元のリストnounsとして保存する）
length = len(lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun

In [151]:
len_n = len(nouns)

In [152]:
nouns_com = [[] for i in range(len_n)]

In [153]:
#分野の関連用語を入力してください
words_field = ['通信', 'ネットワーク', 'スーパーシティ', 'インターネット', 'DX', 'デジタルトランスフォーメーション','サーバー','セキュリティ','データ','コミュニケーション','チャネル','リモート','オンライン', '遠隔医療', 'インフラ']

In [154]:
#処理の軽量化のため、リストの要素のうち必要なもの以外を削除する
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_kigyou:
            nouns_com[i].append(nouns[i][j])
            
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_field:
            nouns_com[i].append(nouns[i][j])

In [155]:
#MeCabで形態素解析を行う
tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text1)
words_wakati1_1 = str_output.split(' ') #元のテキストを分かち書きしたものをwords_wakati1_1に格納する

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text2)
words_wakati1_2 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text3)
words_wakati1_3 = str_output.split(' ')


tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text4)
words_wakati1_4 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text5)
words_wakati1_5 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text6)
words_wakati1_6 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text7)
words_wakati1_7 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text8)
words_wakati1_8 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text9)
words_wakati1_9 = str_output.split(' ')



tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text10)
words_wakati1_10 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text11)
words_wakati1_11 = str_output.split(' ')


tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text12)
words_wakati1_12 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text13)
words_wakati1_13 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text14)
words_wakati1_14 = str_output.split(' ')


tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text15)
words_wakati1_15 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text16)
words_wakati1_16 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text17)
words_wakati1_17 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text18)
words_wakati1_18 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text19)
words_wakati1_19 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text20)
words_wakati1_20 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text21)
words_wakati1_21 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text22)
words_wakati1_22 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text23)
words_wakati1_23 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text24)
words_wakati1_24 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text25)
words_wakati1_25 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text26)
words_wakati1_26 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text27)
words_wakati1_27 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text28)
words_wakati1_28 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text29)
words_wakati1_29 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text30)
words_wakati1_30 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text31)
words_wakati1_31 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text32)
words_wakati1_32 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text33)
words_wakati1_33 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text34)
words_wakati1_34 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text35)
words_wakati1_35 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text36)
words_wakati1_36 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text37)
words_wakati1_37 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text38)
words_wakati1_38 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text39)
words_wakati1_39 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text40)
words_wakati1_40 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text41)
words_wakati1_41 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text42)
words_wakati1_42 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text43)
words_wakati1_43 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text44)
words_wakati1_44 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text45)
words_wakati1_45 = str_output.split(' ')

tagger = MeCab.Tagger("-Owakati")
str_output = tagger.parse(text46)
words_wakati1_46 = str_output.split(' ')

In [156]:
#words_wakati_sup = words_wakati1_1 + words_wakati1_2 + words_wakati1_3 + words_wakati1_4 + words_wakati1_5 + words_wakati1_6 + words_wakati1_7 + words_wakati1_8 + words_wakati1_9

In [157]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_1 = len(words_wakati1_1)

n_2 = len(words_wakati1_2)
n_3 = len(words_wakati1_3)

n_4 = len(words_wakati1_4)

n_5 = len(words_wakati1_5)
n_6 = len(words_wakati1_6)
n_7 = len(words_wakati1_7)
n_8 = len(words_wakati1_8)
n_9 = len(words_wakati1_9)

n_10 = len(words_wakati1_10)
n_11 = len(words_wakati1_11)


n_12 = len(words_wakati1_12)
n_13 = len(words_wakati1_13)
n_14 = len(words_wakati1_14)


n_15 = len(words_wakati1_15)
n_16 = len(words_wakati1_16)
n_17 = len(words_wakati1_17)
n_18 = len(words_wakati1_18)
n_19 = len(words_wakati1_19)
n_20 = len(words_wakati1_20)
n_21 = len(words_wakati1_21)
n_22 = len(words_wakati1_22)
n_23 = len(words_wakati1_23)
n_24 = len(words_wakati1_24)
n_25 = len(words_wakati1_25)
n_26 = len(words_wakati1_26)
n_27 = len(words_wakati1_27)
n_28 = len(words_wakati1_28)
n_29 = len(words_wakati1_29)
n_30 = len(words_wakati1_30)
n_31 = len(words_wakati1_31)
n_32 = len(words_wakati1_32)
n_33 = len(words_wakati1_33)
n_34 = len(words_wakati1_34)
n_35 = len(words_wakati1_35)
n_36 = len(words_wakati1_36)
n_37 = len(words_wakati1_37)
n_38 = len(words_wakati1_38)
n_39 = len(words_wakati1_39)
n_40 = len(words_wakati1_40)
n_41 = len(words_wakati1_41)
n_42 = len(words_wakati1_42)
n_43 = len(words_wakati1_43)
n_44 = len(words_wakati1_44)
n_45 = len(words_wakati1_45)
n_46 = len(words_wakati1_46)

In [158]:
"""
words_wakati1から要素を一つずつ順番に取り出して、その要素がwords_kigyouに含まれているのならば、
新しいリストwords_wakati2にその要素を格納するという作業をwords_wakati1の要素全てに行うことで、
words_wakati2には企業名のみが入ることになる
"""
words_wakati2_1 = []
for i in range(n_1):
    if words_wakati1_1[i] in words_kigyou:
            words_wakati2_1.append(words_wakati1_1[i])
            
words_wakati3_1 = ' '.join(words_wakati2_1)

words_wakati2_2 = []
for i in range(n_2):
    if words_wakati1_2[i] in words_kigyou:
            words_wakati2_2.append(words_wakati1_2[i])
            
words_wakati3_2 = ' '.join(words_wakati2_2)

words_wakati2_3 = []
for i in range(n_3):
    if words_wakati1_3[i] in words_kigyou:
            words_wakati2_3.append(words_wakati1_3[i])
            
words_wakati3_3 = ' '.join(words_wakati2_3)


words_wakati2_4 = []
for i in range(n_4):
    if words_wakati1_4[i] in words_kigyou:
            words_wakati2_4.append(words_wakati1_4[i])
            
words_wakati3_4 = ' '.join(words_wakati2_4)

words_wakati2_5 = []
for i in range(n_5):
    if words_wakati1_5[i] in words_kigyou:
            words_wakati2_5.append(words_wakati1_5[i])
            
words_wakati3_5 = ' '.join(words_wakati2_5)

words_wakati2_6 = []
for i in range(n_6):
    if words_wakati1_6[i] in words_kigyou:
            words_wakati2_6.append(words_wakati1_6[i])
            
words_wakati3_6 = ' '.join(words_wakati2_6)

words_wakati2_7 = []
for i in range(n_7):
    if words_wakati1_7[i] in words_kigyou:
            words_wakati2_7.append(words_wakati1_7[i])
            
words_wakati3_7 = ' '.join(words_wakati2_7)

words_wakati2_8 = []
for i in range(n_8):
    if words_wakati1_8[i] in words_kigyou:
            words_wakati2_8.append(words_wakati1_8[i])
            
words_wakati3_8 = ' '.join(words_wakati2_8)

words_wakati2_9 = []
for i in range(n_9):
    if words_wakati1_9[i] in words_kigyou:
            words_wakati2_9.append(words_wakati1_9[i])
            
words_wakati3_9 = ' '.join(words_wakati2_9)


words_wakati2_10 = []
for i in range(n_10):
    if words_wakati1_10[i] in words_kigyou:
            words_wakati2_10.append(words_wakati1_10[i])
            
words_wakati3_10 = ' '.join(words_wakati2_10)

words_wakati2_11 = []
for i in range(n_11):
    if words_wakati1_11[i] in words_kigyou:
            words_wakati2_11.append(words_wakati1_11[i])
            
words_wakati3_11 = ' '.join(words_wakati2_11)



words_wakati2_12 = []
for i in range(n_12):
    if words_wakati1_12[i] in words_kigyou:
            words_wakati2_12.append(words_wakati1_12[i])
            
words_wakati3_12 = ' '.join(words_wakati2_12)

words_wakati2_13 = []
for i in range(n_13):
    if words_wakati1_13[i] in words_kigyou:
            words_wakati2_13.append(words_wakati1_13[i])
            
words_wakati3_13 = ' '.join(words_wakati2_13)

words_wakati2_14 = []
for i in range(n_14):
    if words_wakati1_14[i] in words_kigyou:
            words_wakati2_14.append(words_wakati1_14[i])
            
words_wakati3_14 = ' '.join(words_wakati2_14)


words_wakati2_15 = []
for i in range(n_15):
    if words_wakati1_15[i] in words_kigyou:
            words_wakati2_15.append(words_wakati1_15[i])
            
words_wakati3_15 = ' '.join(words_wakati2_15)

words_wakati2_16 = []
for i in range(n_16):
    if words_wakati1_16[i] in words_kigyou:
            words_wakati2_16.append(words_wakati1_16[i])
            
words_wakati3_16 = ' '.join(words_wakati2_16)

words_wakati2_17 = []
for i in range(n_17):
    if words_wakati1_17[i] in words_kigyou:
            words_wakati2_17.append(words_wakati1_17[i])
            
words_wakati3_17 = ' '.join(words_wakati2_17)

words_wakati2_18 = []
for i in range(n_18):
    if words_wakati1_18[i] in words_kigyou:
            words_wakati2_18.append(words_wakati1_18[i])
            
words_wakati3_18 = ' '.join(words_wakati2_18)

words_wakati2_19 = []
for i in range(n_19):
    if words_wakati1_19[i] in words_kigyou:
            words_wakati2_19.append(words_wakati1_19[i])
            
words_wakati3_19 = ' '.join(words_wakati2_19)

words_wakati2_20 = []
for i in range(n_20):
    if words_wakati1_20[i] in words_kigyou:
            words_wakati2_20.append(words_wakati1_20[i])
            
words_wakati3_20 = ' '.join(words_wakati2_20)

words_wakati2_21 = []
for i in range(n_21):
    if words_wakati1_21[i] in words_kigyou:
            words_wakati2_21.append(words_wakati1_21[i])
            
words_wakati3_21 = ' '.join(words_wakati2_21)

words_wakati2_22 = []
for i in range(n_22):
    if words_wakati1_22[i] in words_kigyou:
            words_wakati2_22.append(words_wakati1_22[i])
            
words_wakati3_22 = ' '.join(words_wakati2_22)

words_wakati2_23 = []
for i in range(n_23):
    if words_wakati1_23[i] in words_kigyou:
            words_wakati2_23.append(words_wakati1_23[i])
            
words_wakati3_23 = ' '.join(words_wakati2_23)

words_wakati2_24 = []
for i in range(n_24):
    if words_wakati1_24[i] in words_kigyou:
            words_wakati2_24.append(words_wakati1_24[i])
            
words_wakati3_24 = ' '.join(words_wakati2_24)

words_wakati2_25 = []
for i in range(n_25):
    if words_wakati1_25[i] in words_kigyou:
            words_wakati2_25.append(words_wakati1_25[i])
            
words_wakati3_25 = ' '.join(words_wakati2_25)

words_wakati2_26 = []
for i in range(n_26):
    if words_wakati1_26[i] in words_kigyou:
            words_wakati2_26.append(words_wakati1_26[i])
            
words_wakati3_26 = ' '.join(words_wakati2_26)

words_wakati2_27 = []
for i in range(n_27):
    if words_wakati1_27[i] in words_kigyou:
            words_wakati2_27.append(words_wakati1_27[i])
            
words_wakati3_27 = ' '.join(words_wakati2_27)

words_wakati2_28 = []
for i in range(n_28):
    if words_wakati1_28[i] in words_kigyou:
            words_wakati2_28.append(words_wakati1_28[i])
            
words_wakati3_28 = ' '.join(words_wakati2_28)

words_wakati2_29 = []
for i in range(n_29):
    if words_wakati1_29[i] in words_kigyou:
            words_wakati2_29.append(words_wakati1_29[i])
            
words_wakati3_29 = ' '.join(words_wakati2_29)

words_wakati2_30 = []
for i in range(n_30):
    if words_wakati1_30[i] in words_kigyou:
            words_wakati2_30.append(words_wakati1_30[i])
            
words_wakati3_30 = ' '.join(words_wakati2_30)

words_wakati2_31 = []
for i in range(n_31):
    if words_wakati1_31[i] in words_kigyou:
            words_wakati2_31.append(words_wakati1_31[i])
            
words_wakati3_31 = ' '.join(words_wakati2_31)

words_wakati2_32 = []
for i in range(n_32):
    if words_wakati1_32[i] in words_kigyou:
            words_wakati2_32.append(words_wakati1_32[i])
            
words_wakati3_32 = ' '.join(words_wakati2_32)

words_wakati2_33 = []
for i in range(n_33):
    if words_wakati1_33[i] in words_kigyou:
            words_wakati2_33.append(words_wakati1_33[i])
            
words_wakati3_33 = ' '.join(words_wakati2_33)

words_wakati2_34 = []
for i in range(n_34):
    if words_wakati1_34[i] in words_kigyou:
            words_wakati2_34.append(words_wakati1_34[i])
            
words_wakati3_34 = ' '.join(words_wakati2_34)

words_wakati2_35 = []
for i in range(n_35):
    if words_wakati1_35[i] in words_kigyou:
            words_wakati2_35.append(words_wakati1_35[i])
            
words_wakati3_35 = ' '.join(words_wakati2_35)

words_wakati2_36 = []
for i in range(n_36):
    if words_wakati1_36[i] in words_kigyou:
            words_wakati2_36.append(words_wakati1_36[i])
            
words_wakati3_36 = ' '.join(words_wakati2_36)

words_wakati2_37 = []
for i in range(n_37):
    if words_wakati1_37[i] in words_kigyou:
            words_wakati2_37.append(words_wakati1_37[i])
            
words_wakati3_37 = ' '.join(words_wakati2_37)

words_wakati2_38 = []
for i in range(n_38):
    if words_wakati1_38[i] in words_kigyou:
            words_wakati2_38.append(words_wakati1_38[i])
            
words_wakati3_38 = ' '.join(words_wakati2_38)

words_wakati2_39 = []
for i in range(n_39):
    if words_wakati1_39[i] in words_kigyou:
            words_wakati2_39.append(words_wakati1_39[i])
            
words_wakati3_39 = ' '.join(words_wakati2_39)

words_wakati2_40 = []
for i in range(n_40):
    if words_wakati1_40[i] in words_kigyou:
            words_wakati2_40.append(words_wakati1_40[i])
            
words_wakati3_40 = ' '.join(words_wakati2_40)

words_wakati2_41 = []
for i in range(n_41):
    if words_wakati1_41[i] in words_kigyou:
            words_wakati2_41.append(words_wakati1_41[i])
            
words_wakati3_41 = ' '.join(words_wakati2_41)

words_wakati2_42 = []
for i in range(n_42):
    if words_wakati1_42[i] in words_kigyou:
            words_wakati2_42.append(words_wakati1_42[i])
            
words_wakati3_42 = ' '.join(words_wakati2_42)

words_wakati2_43 = []
for i in range(n_43):
    if words_wakati1_43[i] in words_kigyou:
            words_wakati2_43.append(words_wakati1_43[i])
            
words_wakati3_43 = ' '.join(words_wakati2_43)

words_wakati2_44 = []
for i in range(n_44):
    if words_wakati1_44[i] in words_kigyou:
            words_wakati2_44.append(words_wakati1_44[i])
            
words_wakati3_44 = ' '.join(words_wakati2_44)

words_wakati2_45 = []
for i in range(n_45):
    if words_wakati1_45[i] in words_kigyou:
            words_wakati2_45.append(words_wakati1_45[i])
            
words_wakati3_45 = ' '.join(words_wakati2_45)

words_wakati2_46 = []
for i in range(n_46):
    if words_wakati1_46[i] in words_kigyou:
            words_wakati2_46.append(words_wakati1_46[i])
            
words_wakati3_46 = ' '.join(words_wakati2_46)

In [159]:
#リストの結合
#words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14

#通信分野の場合は、おそらく下記のコードを上の代わりに使うことになると思います
words_wakati_WordCloud = words_wakati3_1 + words_wakati3_2 + words_wakati3_3 + words_wakati3_4 + words_wakati3_5 + words_wakati3_6 + words_wakati3_7 + words_wakati3_8 + words_wakati3_9 + words_wakati3_10 + words_wakati3_11 + words_wakati3_12 + words_wakati3_13 + words_wakati3_14 + words_wakati3_15 + words_wakati3_16 + words_wakati3_17 + words_wakati3_18 + words_wakati3_19 + words_wakati3_20 + words_wakati3_21 + words_wakati3_22 + words_wakati3_23 + words_wakati3_24 + words_wakati3_25 + words_wakati3_26 + words_wakati3_27 + words_wakati3_28 + words_wakati3_29 + words_wakati3_30 + words_wakati3_31 + words_wakati3_32 + words_wakati3_33 + words_wakati3_34 + words_wakati3_35 + words_wakati3_36 + words_wakati3_37 + words_wakati3_38 + words_wakati3_39 + words_wakati3_40 + words_wakati3_41 + words_wakati3_42 + words_wakati3_43 + words_wakati3_44 + words_wakati3_45 + words_wakati3_46

In [179]:
cd /Users/shintaro/Downloads/第1スクリーニング

/Users/shintaro/Downloads/第1スクリーニング


In [180]:
#WordCloudを出力したい場合は最終行の#を消去してください
stop_words = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース'] #wordcloudの結果に入れたくないwordを決定する
fpath = '/Users/shintaro/Library/Fonts//Arial Unicode.ttf'  # 日本語フォント指定
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    max_words = 500,   # default=200
    min_font_size = 4,   #default=4
    prefer_horizontal = 1, 
    colormap = 'tab20',
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

wordcloud.to_file("WordCloud_通信.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [162]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [163]:
#words_wakati_WordCloud1から同一名称の成分を除外したリストwwW_uniqueを作成する
wwW_unique = list(set(words_wakati_WordCloud1))
#wwW_unique

In [164]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c5 = c.most_common()

In [165]:
#単語２つの組み合わせを作成
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [166]:
#リスト内の要素の整理
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [167]:
#上で作成したcombination_sentencesは２次元リストなので１次元リストに変換する
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [168]:
combination_sentences = tmp

In [169]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns=['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns=['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [170]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'w', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [171]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [172]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [173]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [174]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [181]:
jaccard_coef_data2.to_csv("jaccard_coef_通信.csv", index = False, encoding = 'utf-8-sig')

## 第1スクリーニング　最終選定

In [182]:
#日経NEEDSでの分野分けとテキストマイニングで算出したジャッカード係数の結果を合わせて第一スクリーニングを行う

In [183]:
#各分野のjaccard係数の結果が記載されたファイルを読み込む

In [184]:
import csv
import pandas as pd
import collections

In [185]:
cd /Users/shintaro/Downloads/textmining-medical

/Users/shintaro/Downloads/textmining-medical


In [186]:
jc_m = pd.read_csv(filepath_or_buffer = "jaccard_coef_medical.csv", encoding = 'utf8', sep = ",", usecols = ['word1', 'word2'])
jc_m_ = pd.read_csv(filepath_or_buffer = "jaccard_coef_medical.csv", encoding = 'utf8', sep = ",")

In [187]:
cd /Users/shintaro/Downloads/textmining-care

/Users/shintaro/Downloads/textmining-care


In [188]:
jc_m_

word1             word2  intersection_count  count1  count2  \
0            四国銀行              高知銀行                   4       4       4   
1   メディパルホールディングス            楽天グループ                   2       2       2   
2         ティーケーピー  ユニバーサルエンターテインメント                   2       2       2   
3     アイ・オー・データ機器              三谷産業                   2       2       2   
4           アサカ理研              鉱研工業                   1       1       1   
..            ...               ...                 ...     ...     ...   
90           商船三井              川崎汽船                   2      29       9   
91         住友電気工業    近鉄グループホールディングス                   1       9      10   
92         住友電気工業      阪急阪神ホールディングス                   1       9      10   
93           オムロン             キーエンス                  12     184      48   
94         ヤクルト本社           大日本住友製薬                   1       4      16   

    union_count  jaccard_coef  
0             4      1.000000  
1             2      1.000000  
2             2      1.000000  
3             2      1.000000  
4             1      1.000000  
..          ...           ...  
90           36      0.055556  
91           18      0.055556  
92           18      0.055556  
93          220      0.054545  
94           19      0.052632  

[95 rows x 7 columns]

In [189]:
c1

[('オリンパス', 435),
 ('キヤノン', 284),
 ('シスメックス', 173),
 ('東芝', 168),
 ('ニプロ', 163),
 ('テルモ', 163),
 ('コニカミノルタ', 145),
 ('オムロン', 144),
 ('富士通', 137),
 ('日機装', 125),
 ('リコー', 124),
 ('島津製作所', 121),
 ('シャープ', 109),
 ('パナソニック', 102),
 ('エーザイ', 99),
 ('帝人', 88),
 ('日立製作所', 82),
 ('アステラス製薬', 77),
 ('ソフトバンク', 77),
 ('鈴木', 74),
 ('ニコン', 69),
 ('イオン', 68),
 ('富士フイルムホールディングス', 65),
 ('塩野義製薬', 62),
 ('日医工', 62),
 ('フリー', 57),
 ('凸版印刷', 54),
 ('エムスリー', 52),
 ('川崎重工業', 51),
 ('デンカ', 51),
 ('トヨタ自動車', 48),
 ('コーセル', 48),
 ('アンジェス', 45),
 ('ハブ', 44),
 ('花王', 42),
 ('岩谷産業', 40),
 ('菊池製作所', 40),
 ('ソニーグループ', 39),
 ('村田製作所', 38),
 ('昭和電工', 38),
 ('マニー', 38),
 ('メニコン', 37),
 ('第一三共', 36),
 ('オークネット', 36),
 ('アルファ', 36),
 ('日本トリム', 36),
 ('エア・ウォーター', 35),
 ('ＴＤＫ', 35),
 ('メドピア', 35),
 ('ズーム', 34),
 ('東レ', 33),
 ('クボタ', 32),
 ('住友ベークライト', 31),
 ('三菱電機', 30),
 ('朝日インテック', 30),
 ('アシックス', 30),
 ('日新', 29),
 ('安川電機', 29),
 ('シークス', 29),
 ('アウトソーシング', 28),
 ('ソフトバンクグループ', 27),
 ('ＳＭＣ', 27),
 ('住友商事', 27),
 ('グンゼ', 

In [190]:
jc_c = pd.read_csv(filepath_or_buffer = "jaccard_coef_care.csv", encoding = 'utf8', sep = ",", usecols = ['word1', 'word2'])

In [191]:
cd /Users/shintaro/Downloads

/Users/shintaro/Downloads


In [192]:
jc_モビリティ = pd.read_csv(filepath_or_buffer = "jaccardモビリティ改.csv", encoding = 'utf8', sep = ",", usecols = ['word1', 'word2'])

In [193]:
jc_運送 = pd.read_csv(filepath_or_buffer = "jaccard運送改.csv", encoding = 'utf8', sep = ",", usecols = ['word1', 'word2'])

In [194]:
jc_通信 = pd.read_csv(filepath_or_buffer = "jaccard_coef_通信.csv", encoding = 'utf8', sep = ",", usecols = ['word1', 'word2'])

In [195]:
jc_m1 = jc_m['word1'].tolist()
jc_m2 = jc_m['word2'].tolist()

In [196]:
jc_m_all = jc_m1 + jc_m2

In [197]:
jc_c1 = jc_c['word1'].tolist()
jc_c2 = jc_c['word2'].tolist()

In [198]:
jc_c_all = jc_c1 + jc_c2

In [199]:
jc_モビリティ1 = jc_モビリティ['word1'].tolist()
jc_モビリティ2 = jc_モビリティ['word2'].tolist()

In [200]:
jc_モビリティ_all = jc_モビリティ1 + jc_モビリティ2

In [201]:
jc_運送1 = jc_運送['word1'].tolist()
jc_運送2 = jc_運送['word2'].tolist()

In [202]:
jc_運送_all = jc_運送1 + jc_運送2

In [203]:
jc_通信1 = jc_通信['word1'].tolist()
jc_通信2 = jc_通信['word2'].tolist()

In [204]:
jc_通信_all = jc_通信1 + jc_通信2

In [205]:
jc_All = jc_m_all + jc_c_all + jc_モビリティ_all + jc_運送_all + jc_通信_all

In [206]:
#jaccard係数によって残った企業のリスト
jc_All_tmp = jc_All

In [209]:
#jc_Allには一部企業名ではない要素と重複があるので以下で要素を整理する
pickup_words = ['通信', 'ネットワーク', 'スーパーシティ', 'インターネット', 'DX', 'デジタルトランスフォーメーション','サーバー','セキュリティ','データ','コミュニケーション','チャネル','リモート','オンライン']
jc_All = []
pickup_words_jc = []
for i in range(len(jc_All_tmp)):
    if jc_All_tmp[i] in pickup_words:
        pickup_words_jc.append(jc_All_tmp[i])
    else:
        jc_All.append(jc_All_tmp[i])
        
jc_All_unique = list(set(jc_All))

jc_All_unique_tmp = jc_All_unique
jc_All_unique = []

#ワードクラウドのstop_wordsのように、該当しない企業名を削除する
false_company = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース']
false_company_jc = []
for i in range(len(jc_All_unique_tmp)):
    if jc_All_unique_tmp[i] in false_company:
        false_company_jc.append(jc_All_unique_tmp[i])
    else:
        jc_All_unique.append(jc_All_unique_tmp[i])

In [210]:
cd /Users/shintaro/Downloads

/Users/shintaro/Downloads


In [211]:
#日経NEEDSでの分野分けされた企業が入ったリストを結合する
com1_tmp = pd.read_csv("care.csv", encoding = "utf-8")
com1_tmp1 = com1_tmp["銘柄名称"].tolist()

com1 = []

for i in range(len(com1_tmp1)):
    com1.append(com1_tmp1[i].replace('\u3000', ''))

com2_tmp = pd.read_csv("medical.csv", encoding = "utf-8")
com2_tmp1 = com2_tmp["銘柄名称"].tolist()

com2 = []

for i in range(len(com2_tmp1)):
    com2.append(com2_tmp1[i].replace('\u3000', ''))
    
com3_tmp = pd.read_csv("モビリティ銘柄.csv", encoding = "utf-8")
com3_tmp1 = com3_tmp["銘柄名称"].tolist()

com3 = []

for i in range(len(com3_tmp1)):
    com3.append(com3_tmp1[i].replace('\u3000', ''))
    
com4_tmp = pd.read_csv("運送銘柄.csv", encoding = "utf-8")
com4_tmp1 = com4_tmp["銘柄名称"].tolist()

com4 = []

for i in range(len(com4_tmp1)):
    com4.append(com4_tmp1[i].replace('\u3000', ''))

com5_tmp = pd.read_csv("nst_0827141430.csv", encoding = "utf-8")
com5_tmp1 = com5_tmp["銘柄名称"].tolist()

com5 = []

for i in range(len(com5_tmp1)):
    com5.append(com5_tmp1[i].replace('\u3000', ''))
    
com_All = com1 + com2 + com3 + com4 + com5 + jc_All_unique

#重複のないリストにする
com_fstscreening1 = list(set(com_All))

#このcom_fstscreening1が第一スクリーニングで残った企業のリストになります

In [ ]:
#結果をcsvファイルに書き込む
with open("fst.screening1.csv", "w", encoding="utf-8-sig") as f: 
    writer = csv.writer(f)
    writer.writerow(com_fstscreening1)

In [212]:
len(com_fstscreening1)

808

# 上の方法ではうまくスクリーニングできていないので、別の方法で行う

In [214]:
c1_tmp = []

for i in range(len(c1)):
    if c1[i][1] >= 10:
        c1_tmp.append(c1[i])

c2_tmp = []

for i in range(len(c2)):
    if c2[i][1] >= 10:
        c2_tmp.append(c2[i])

c3_tmp = []

for i in range(len(c3)):
    if c3[i][1] >= 10:
        c3_tmp.append(c3[i])
        
c4_tmp = []

for i in range(len(c4)):
    if c4[i][1] >= 10:
        c4_tmp.append(c4[i])
        
c5_tmp = []

for i in range(len(c5)):
    if c5[i][1] >= 10:
        c5_tmp.append(c5[i])

In [215]:
c1_tmp1 = []
for i in range(len(c1_tmp)):
    c1_tmp1.append(c1_tmp[i][0])
    
c2_tmp1 = []
for i in range(len(c2_tmp)):
    c2_tmp1.append(c2_tmp[i][0])
    
c3_tmp1 = []
for i in range(len(c3_tmp)):
    c3_tmp1.append(c3_tmp[i][0])
    
c4_tmp1 = []
for i in range(len(c4_tmp)):
    c4_tmp1.append(c4_tmp[i][0])
    
c5_tmp1 = []
for i in range(len(c5_tmp)):
    c5_tmp1.append(c5_tmp[i][0])

In [216]:
jc_m_tmp = [[] for i in range(len(jc_m1))]

for i in range(len(jc_m_tmp)):
    jc_m_tmp[i].append(jc_m1[i])
    jc_m_tmp[i].append(jc_m2[i])
    
jc_c_tmp = [[] for i in range(len(jc_c1))]

for i in range(len(jc_c_tmp)):
    jc_c_tmp[i].append(jc_c1[i])
    jc_c_tmp[i].append(jc_c2[i])
    
jc_モビリティ_tmp = [[] for i in range(len(jc_モビリティ1))]

for i in range(len(jc_モビリティ_tmp)):
    jc_モビリティ_tmp[i].append(jc_モビリティ1[i])
    jc_モビリティ_tmp[i].append(jc_モビリティ2[i])
    
jc_運送_tmp = [[] for i in range(len(jc_運送1))]

for i in range(len(jc_運送_tmp)):
    jc_運送_tmp[i].append(jc_運送1[i])
    jc_運送_tmp[i].append(jc_運送2[i])
    
jc_通信_tmp = [[] for i in range(len(jc_通信1))]

for i in range(len(jc_通信_tmp)):
    jc_通信_tmp[i].append(jc_通信1[i])
    jc_通信_tmp[i].append(jc_通信2[i])

In [217]:
c1_m1 = []

for i in range(len(jc_m_tmp)):
    if jc_m_tmp[i][0] in c1_tmp1 or jc_m_tmp[i][1] in c1_tmp1:
        c1_m1.append(jc_m_tmp[i][0])
        c1_m1.append(jc_m_tmp[i][1])
        
c2_c1 = []

for i in range(len(jc_c_tmp)):
    if jc_c_tmp[i][0] in c2_tmp1 or jc_c_tmp[i][1] in c2_tmp1:
        c2_c1.append(jc_c_tmp[i][0])
        c2_c1.append(jc_c_tmp[i][1])
        
c3_モビリティ1 = []

for i in range(len(jc_モビリティ_tmp)):
    if jc_モビリティ_tmp[i][0] in c3_tmp1 or jc_モビリティ_tmp[i][1] in c3_tmp1:
        c3_モビリティ1.append(jc_モビリティ_tmp[i][0])
        c3_モビリティ1.append(jc_モビリティ_tmp[i][1])

c4_運送1 = []

for i in range(len(jc_運送_tmp)):
    if jc_運送_tmp[i][0] in c4_tmp1 or jc_運送_tmp[i][1] in c4_tmp1:
        c4_運送1.append(jc_運送_tmp[i][0])
        c4_運送1.append(jc_運送_tmp[i][1])
        
c5_通信1 = []

for i in range(len(jc_通信_tmp)):
    if jc_通信_tmp[i][0] in c5_tmp1 or jc_通信_tmp[i][1] in c5_tmp1:
        c5_通信1.append(jc_通信_tmp[i][0])
        c5_通信1.append(jc_通信_tmp[i][1])

In [218]:
c_All1 = c1_m1 + c2_c1 + c3_モビリティ1 + c4_運送1 + c5_通信1

In [219]:
c1_m2 = []
c2_c2 = []
c3_モビリティ2 = []
c4_運送2 = []
c5_通信2 = []

for i in range(len(jc_m_tmp)):
    if jc_m_tmp[i][0] in com1 or jc_m_tmp[i][1] in com1:
        c1_m2.append(jc_m_tmp[i][0])
        c1_m2.append(jc_m_tmp[i][1])

for i in range(len(jc_c_tmp)):
    if jc_c_tmp[i][0] in com2 or jc_c_tmp[i][1] in com2:
        c2_c2.append(jc_c_tmp[i][0])
        c2_c2.append(jc_c_tmp[i][1])

for i in range(len(jc_モビリティ_tmp)):
    if jc_モビリティ_tmp[i][0] in com3 or jc_モビリティ_tmp[i][1] in com3:
        c3_モビリティ2.append(jc_モビリティ_tmp[i][0])
        c3_モビリティ2.append(jc_モビリティ_tmp[i][1])

for i in range(len(jc_運送_tmp)):
    if jc_運送_tmp[i][0] in com4 or jc_運送_tmp[i][1] in com4:
        c4_運送2.append(jc_運送_tmp[i][0])
        c4_運送2.append(jc_運送_tmp[i][1])
    
for i in range(len(jc_通信_tmp)):
    if jc_通信_tmp[i][0] in com5 or jc_通信_tmp[i][1] in com5:
        c5_通信2.append(jc_通信_tmp[i][0])
        c5_通信2.append(jc_通信_tmp[i][1])

In [220]:
c_All2 = c1_m2 + c2_c2 + c3_モビリティ2 + c4_運送2 + c5_通信2

In [221]:
c_ALL = c_All1 + c_All2

In [222]:
for i in range(20):
    c_ALL.append(c1[i][0])
    c_ALL.append(c2[i][0])
    c_ALL.append(c3[i][0])
    c_ALL.append(c4[i][0])
    c_ALL.append(c5[i][0])

In [223]:
jc_m_tmp[0][1]

'高知銀行'

In [224]:
pickup_words1 = ['医療', 'AI', '介護', 'ロボット', 'スーパーシティ', '遠隔医療', 'DX', 'デジタルトランスフォーメーション']

for i in range (len(jc_m_tmp)):
    if jc_m_tmp[i][0] in pickup_words1 or jc_m_tmp[i][1] in pickup_words1:
        c_ALL.append(jc_m_tmp[i][0])
        c_ALL.append(jc_m_tmp[i][1])
        
pickup_words2 = ['介護', 'AI', 'ロボット', 'スーパーシティ', 'DX', 'デジタルトランスフォーメーション']

for i in range (len(jc_c_tmp)):
    if jc_c_tmp[i][0] in pickup_words2 or jc_c_tmp[i][1] in pickup_words2:
        c_ALL.append(jc_c_tmp[i][0])
        c_ALL.append(jc_c_tmp[i][1])
        
pickup_words3 = ['モビリティ', 'AI', 'スーパーシティ', 'DX', 'デジタルトランスフォーメーション']

for i in range (len(jc_モビリティ_tmp)):
    if jc_モビリティ_tmp[i][0] in pickup_words3 or jc_モビリティ_tmp[i][1] in pickup_words3:
        c_ALL.append(jc_モビリティ_tmp[i][0])
        c_ALL.append(jc_モビリティ_tmp[i][1])
        
pickup_words4 = ['運送', 'AI', 'スーパーシティ', 'DX', 'デジタルトランスフォーメーション']

for i in range (len(jc_運送_tmp)):
    if jc_運送_tmp[i][0] in pickup_words4 or jc_運送_tmp[i][1] in pickup_words4:
        c_ALL.append(jc_運送_tmp[i][0])
        c_ALL.append(jc_運送_tmp[i][1])
        
pickup_words5 = ['通信', 'ネットワーク', 'スーパーシティ', 'インターネット', 'DX', 'デジタルトランスフォーメーション','サーバー','セキュリティ','データ','コミュニケーション','チャネル','リモート','オンライン', 'インフラ']

for i in range (len(jc_通信_tmp)):
    if jc_通信_tmp[i][0] in pickup_words5 or jc_通信_tmp[i][1] in pickup_words5:
        c_ALL.append(jc_通信_tmp[i][0])
        c_ALL.append(jc_通信_tmp[i][1])

In [225]:
c_ALL_unique = list(set(c_ALL))
c_all = c_ALL_unique
c_all_tmp = c_all

In [262]:
#c_allには一部企業名ではない要素と重複があるので以下で要素を整理する
pickup_words = ['モビリティ', '自動車', '運送', 'AI', '医療', '介護', 'ロボット', '遠隔医療', '通信', 'ネットワーク', 'スーパーシティ', 'インターネット', 'DX', 'デジタルトランスフォーメーション','サーバー','セキュリティ','データ','コミュニケーション','チャネル','リモート','オンライン', 'インフラ', 'ソニーグループ']
c_all = []
pickup_words_c = []
for i in range(len(c_all_tmp)):
    if c_all_tmp[i] in pickup_words:
        pickup_words_c.append(c_all_tmp[i])
    else:
        c_all.append(c_all_tmp[i])
        
c_all_tmp = c_all
comp_all = []

#ワードクラウドのstop_wordsのように、実在しない企業名を削除する
false_company = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース']
false_company_c = []
for i in range(len(c_all_tmp)):
    if c_all_tmp[i] in false_company:
        false_company_c.append(c_all_tmp[i])
    else:
        comp_all.append(c_all_tmp[i])
        
com_fstscreening2 = comp_all

In [263]:
#結果をcsvファイルに書き込む
with open("fst.screening2(ver1.1).csv", "w", encoding="utf-8-sig") as f: 
    writer = csv.writer(f)
    writer.writerow(com_fstscreening2)

In [270]:
len(com_fstscreening2)

219